**Pendahuluan**

Halo semuanya, setelah saya berusaha untuk menyelesaikan kelas belajar pengembangan machine learning dari dicoding indonesia sambil melakukan kesibukan lain seperti menyusun skripsi dan hal lainnya. Akhirnya saya berhasil menyelesaikan submission pertama yaitu membuat model NLP dengan TenserFlow. Akurasi yang harus didapatkan agar lulus submission pertama ini adalah 75%.

Pertama kita import terlebih dahulu library yang akan kita gunakan.

In [ ]:
# import pandas
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import matplotlib.pyplot as plt

Kemudian import file yang akan kita gunakan, disini saya menggunakan dataset tentang berita dari BBC News.

Tampilkan 5 data teratas.

In [ ]:
datasetnya = pd.read_csv('../input/bbcnewsarchive/bbc-news-data.csv', sep='\t')
datasetnya.head()

Kemudian drop data yang tidak akan kita gunakan.

In [ ]:
new_dataset = datasetnya.drop(columns=['filename'])
new_dataset.head()

**Model dan Plot**

In [ ]:
kategori = pd.get_dummies(new_dataset.category)
newest_dataset = pd.concat([new_dataset, kategori], axis=1)
newest_dataset = newest_dataset.drop(columns='category')
newest_dataset

Ubah nilai pada dataframe menjadi sebuah numpy array

In [ ]:

bbc_news = newest_dataset['title'].values + '' + newest_dataset['content'].values
bbc_berita = newest_dataset[['business', 'entertainment', 'politics', 'sport', 'tech']].values

Melihat isi array

In [ ]:
bbc_news

Melihat label array

In [ ]:
bbc_berita

Split data menjadi data training dan data validation

In [ ]:
bbc_news_train, bbc_news_test, bbc_berita_train, bbc_berita_test = train_test_split(bbc_news, bbc_berita, test_size=0.2)

Proses tokenizer

In [ ]:
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(bbc_news_train) 
tokenizer.fit_on_texts(bbc_news_test)
 
sekuens_train = tokenizer.texts_to_sequences(bbc_news_train)
sekuens_test = tokenizer.texts_to_sequences(bbc_news_test)
 
padded_train = pad_sequences(sekuens_train) 
padded_test = pad_sequences(sekuens_test)

Model

In [ ]:
model = tf.keras.Sequential([
     tf.keras.layers.Embedding(input_dim=5000, output_dim=64),
     tf.keras.layers.BatchNormalization(),
     tf.keras.layers.LSTM(128),
     tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dropout(0.5),
     tf.keras.layers.Dense(5, activation='softmax')
     ])
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])
model.summary()

Callback

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.75 and logs.get('val_acc')>0.9):
      self.model.stop_training = True
      print("\nAkurasinya > 75%!")

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)
callbacks = myCallback()

Proses fit model

In [ ]:
history = model.fit(padded_train, bbc_berita_train, epochs=50, 
                    validation_data=(padded_test, bbc_berita_test), verbose=2, callbacks=[callbacks], validation_steps=30)

Model Akurasi

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Model Loss

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()